In [1]:
import pandas as pd
import numpy as np
import re
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /Users/st_dare/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/st_dare/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/st_dare/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/st_dare/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/st_dare/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
train = pd.read_csv("Data/processed/train.csv")
test = pd.read_csv("Data/processed/test.csv")

# Applying our post inspection changes
train.dropna(inplace=True)
test.dropna(inplace=True)

display(train.head(), test.head())

,headlines,description,content,url,category
0,RBI revises definition of politically-exposed ...,The central bank has also asked chairpersons a...,The Reserve Bank of India (RBI) has changed th...,https://indianexpress.com/article/business/ban...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,NDTV's consolidated revenue from operations wa...,Broadcaster New Delhi Television Ltd on Monday...,https://indianexpress.com/article/business/com...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",The initial share sale will be open for public...,Homegrown server maker Netweb Technologies Ind...,https://indianexpress.com/article/business/mar...,business
3,India’s current account deficit declines sharp...,The current account deficit (CAD) was 3.8 per ...,India’s current account deficit declined sharp...,https://indianexpress.com/article/business/eco...,business
4,"States borrowing cost soars to 7.68%, highest ...",The prices shot up reflecting the overall high...,States have been forced to pay through their n...,https://indianexpress.com/article/business/eco...,business


,headlines,description,content,url,category
0,NLC India wins contract for power supply to Ra...,State-owned firm NLC India Ltd (NLCIL) on Mond...,State-owned firm NLC India Ltd (NLCIL) on Mond...,https://indianexpress.com/article/business/com...,business
1,SBI Clerk prelims exams dates announced; admit...,SBI Clerk Prelims Exam: The SBI Clerk prelims ...,SBI Clerk Prelims Exam: The State Bank of Indi...,https://indianexpress.com/article/education/sb...,education
2,"Golden Globes: Michelle Yeoh, Will Ferrell, An...","Barbie is the top nominee this year, followed ...","Michelle Yeoh, Will Ferrell, Angela Bassett an...",https://indianexpress.com/article/entertainmen...,entertainment
3,"OnePlus Nord 3 at Rs 27,999 as part of new pri...",New deal makes the OnePlus Nord 3 an easy purc...,"In our review of the OnePlus Nord 3 5G, we pra...",https://indianexpress.com/article/technology/t...,technology
4,Adani family’s partners used ‘opaque’ funds to...,Citing review of files from multiple tax haven...,Millions of dollars were invested in some publ...,https://indianexpress.com/article/business/ada...,business


In [3]:
train['data'] = train['headlines'] + train['category'] + train['content']
test['data'] = test['headlines'] + test['category'] + test['content'] 

train = train[['data', 'category']]
test = test[['data', 'category']]

display(train.head(), test.head())

,data,category
0,RBI revises definition of politically-exposed ...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",business
3,India’s current account deficit declines sharp...,business
4,"States borrowing cost soars to 7.68%, highest ...",business


,data,category
0,NLC India wins contract for power supply to Ra...,business
1,SBI Clerk prelims exams dates announced; admit...,education
2,"Golden Globes: Michelle Yeoh, Will Ferrell, An...",entertainment
3,"OnePlus Nord 3 at Rs 27,999 as part of new pri...",technology
4,Adani family’s partners used ‘opaque’ funds to...,business


In [4]:
# 1. Improved Preprocessing Function

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def process_text_pro(text: str) -> str:
    # 1. Lowercase
    text = text.lower()
    # 2. Remove URLs, mentions, hashtags
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+", " ", text)
    # 3. Remove non-alphabetic characters (keep spaces)
    text = re.sub(r'[^a-z\s]', ' ', text)
    # 4. Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # 5. Tokenize
    tokens = word_tokenize(text)
    # 6. Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # 7. Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

train['data'] = train['data'].apply(process_text_pro)
test['data'] = test['data'].apply(process_text_pro)

In [5]:
X_train = train['data']
y_train = train['category']

X_test = test['data']
y_test = test['category']


# Initialise TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # Unigrams and bigrams; adjust max_features as needed

# Fit and transform the text data
X_tfidf = tfidf_vectorizer.fit_transform(X_train)
Xt_tfidf = tfidf_vectorizer.transform(X_test)

# Display the shape of the resulting sparse matrix
print(f"TF-IDF Matrix Shape: {X_tfidf.shape}")

# Display the vocabulary (optional)
print("Vocabulary:", tfidf_vectorizer.get_feature_names_out())

# Initialise the Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model on the training set
model.fit(X_tfidf, y_train)

# Print training accuracy
train_accuracy = model.score(X_tfidf, y_train)
print(f"Training Accuracy: {train_accuracy:.2f}")

# Make predictions on the validation set
y_pred = model.predict(Xt_tfidf)

# Compute validation accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {test_accuracy:.2f}")

# Generate a classification report
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=y_test.unique()))

TF-IDF Matrix Shape: (5520, 5000)
Vocabulary: ['aadhaar' 'aamir' 'aamir khan' ... 'zoom' 'zoya' 'zoya akhtar']
Training Accuracy: 0.99
Validation Accuracy: 0.98
Classification Report:
                precision    recall  f1-score   support

     business       0.98      0.96      0.97       400
    education       1.00      0.99      0.99       400
entertainment       1.00      0.98      0.99       400
   technology       0.99      0.98      0.98       400
       sports       0.93      0.98      0.96       400

     accuracy                           0.98      2000
    macro avg       0.98      0.98      0.98      2000
 weighted avg       0.98      0.98      0.98      2000



In [6]:
import pickle

In [7]:
import pickle

with open('pickled_files/model_and_vectorizer.pkl', 'wb') as f:
    pickle.dump({'model': model, 'vectorizer': tfidf_vectorizer}, f)

In [8]:
!streamlit run Streamlit/base_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.5:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
^C


In [9]:
input_text = input("Enter your text here: ")

In [16]:
pt= process_text_pro(input_text)
pt

'feel happy today everything seems brighter stop smiling one day little thing make difference'

In [17]:
vect_t = tfidf_vectorizer.transform([pt])

In [18]:
vect_t

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 13 stored elements and shape (1, 5000)>

In [19]:
model.predict(vect_t)

array(['technology'], dtype=object)

In [ ]:
breaker = True

while breaker:
    input_text = input("Enter Text to Classify, Enter End/Quit to end program")

    if input_text.lower() == "end" or input_text.lower() == "quit":
        break
    else:
        pt = process_text_pro(input_text)
        vect_t = tfidf_vectorizer.transform([pt])
        p_class = model.predict(vect_t)
        print(p_class)



['sports']
['education']
